## Importing the dependencies


In [ ]:
# Sierra Leone Solar Data Analysis
# EDA for Sierra Leone solar dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

## Loading the files

In [ ]:
## Data Loading
print("=== LOADING SIERRA LEONE DATA ===")

df = pd.read_csv('../data/sierraleone-bumbuna.csv')
print(f"Data shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\nFirst 5 rows:")
print(df.head())

## Summary Statistics & Missing Values

In [ ]:
print("An overview of the dataset: ")
print(df.info())

In [ ]:
# Summary Statistics for all numeric columns
print("The description of the numeric columns:")
print(df.describe())

In [ ]:
#Overview of categorical columns
print("The description of the categorical columns:")
print(df.describe(include=['object']))

In [ ]:
print("The description of all columns:")
print(df.describe(include='all'))

In [ ]:
# Missing Values Analysis using also a percent to use it for later on Z-score
print("\n MISSING VALUES REPORT ")
missing_data = df.isna().sum()
missing_percent = (df.isna().sum() / len(df)) * 100

missing_report = pd.DataFrame({
    'Missing_Count': missing_data,
    'Missing_Percent': missing_percent
})

print(missing_report)

In [ ]:
# Exact duplicate rows
dup_count = df.duplicated().sum()
print("Duplicate rows:", dup_count)

# Cardinality (uniqueness) for categoricals
cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
cardinality = {c: df[c].nunique() for c in cat_cols}
print("Cardinality (categoricals):", cardinality)


In [ ]:
# Columns with more than 5% missing values
print("COLUMNS WITH >5% MISSING VALUES")
high_missing = missing_report[missing_report['Missing_Percent'] > 5]

if not high_missing.empty:
    print("Missing values:")
    print(high_missing)
else:
    print("No columns have more than 5% missing values")

# Outlier detection and Basic Cleaning

In [ ]:
# Define key columns for outlier detection
key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
print("=== ANALYZING KEY COLUMNS FOR OUTLIERS ===")
print("Key columns:", key_columns)

# Check which of these columns exist in our data
available_columns = [col for col in key_columns if col in df.columns]
print("Available columns in dataset:", available_columns)

In [ ]:
# Compute Z-scores and flag outliers (|Z| > 3)
print("\nOUTLIER DETECTION USING Z-SCORES")

# Create a copy of the dataframe for cleaning
df_clean = df.copy()
key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
available_columns = [col for col in key_columns if col in df.columns]
for column in available_columns:
    # Calculate Z-scores
    z_scores = np.abs(stats.zscore(df_clean[column].dropna()))
    
    # Find outliers
    outliers = z_scores > 3
    outlier_count = outliers.sum()
    outlier_percent = (outlier_count / len(df_clean[column].dropna())) * 100
    
    print(f"{column}: {outlier_count} outliers ({outlier_percent:.2f}%)")
    
    # Flag outliers in the dataframe
    df_clean[f'{column}_outlier'] = False
    df_clean.loc[df_clean[column].notna(), f'{column}_outlier'] = outliers

# Total rows with any outlier
any_outlier = df_clean[[f'{col}_outlier' for col in available_columns]].any(axis=1)
print(f"\nTotal rows with at least one outlier: {any_outlier.sum()} ({any_outlier.mean()*100:.2f}%)")

In [ ]:
# Drop or impute missing values in key columns
print("\n MISSING VALUE IMPUTE")

for column in available_columns:
    missing_before = df_clean[column].isna().sum()
    
    if missing_before > 0:
        # Impute with median (you can change to mean if preferred)
        median_value = df_clean[column].median()
        df_clean[column].fillna(median_value, inplace=True)
        
        missing_after = df_clean[column].isna().sum()
        print(f"{column}: Imputed {missing_before} missing values with median {median_value:.2f}")

print("\nMissing values after treatment:")
print(df_clean[available_columns].isna().sum())

In [ ]:
## Export Cleaned Data
print("=== EXPORTING CLEANED DATA ===")

df_clean.to_csv('../data/sierraleone_clean.csv', index=False)
print("Cleaned data saved to: ../data/sierraleone_clean.csv")

# Time series analysis

In [ ]:
## Time Series Analysis
print("=== TIME SERIES ANALYSIS ===")

# Convert timestamp
df_clean['Timestamp'] = pd.to_datetime(df_clean['Timestamp'])
df_clean['Hour'] = df_clean['Timestamp'].dt.hour
df_clean['Month'] = df_clean['Timestamp'].dt.month

# Plot time series for key variables
variables_to_plot = ['GHI', 'DNI', 'DHI', 'Tamb']

for var in variables_to_plot:
    if var in df_clean.columns:
        plt.figure(figsize=(12, 4))
        # Plot first 500 points for clarity
        plt.plot(df_clean['Timestamp'].iloc[:500], df_clean[var].iloc[:500])
        plt.title(f'{var} Over Time (First 500 points)')
        plt.xlabel('Timestamp')
        plt.ylabel(var)
        plt.xticks(rotation=45)
        plt.grid(True, alpha=0.3)
        plt.show()

In [ ]:
## Daily Patterns
print("=== DAILY PATTERNS ===")

# Calculate hourly averages
hourly_avg = df_clean.groupby('Hour')[variables_to_plot].mean()

plt.figure(figsize=(12, 6))
for var in variables_to_plot:
    if var in hourly_avg.columns:
        plt.plot(hourly_avg.index, hourly_avg[var], marker='o', label=var)

plt.title('Average Daily Patterns by Hour')
plt.xlabel('Hour of Day')
plt.ylabel('Values')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(range(0, 24, 2))
plt.show()

In [ ]:
## Monthly Patterns
print("=== MONTHLY PATTERNS ===")

monthly_avg = df_clean.groupby('Month')[variables_to_plot].mean()

plt.figure(figsize=(12, 6))
for var in variables_to_plot:
    if var in monthly_avg.columns:
        plt.plot(monthly_avg.index, monthly_avg[var], marker='s', label=var)

plt.title('Average Monthly Patterns')
plt.xlabel('Month')
plt.ylabel('Values')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xticks(range(1, 13))
plt.show()

In [ ]:
# Detect potential anomalies
print("=== ANOMALY DETECTION ===")

# Find days with unusual patterns (e.g., very low GHI during daytime)
daytime_hours = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
daytime_data = df_clean[df_clean['Hour'].isin(daytime_hours)]

# Flag days with very low solar radiation during daytime
low_ghi_days = daytime_data[daytime_data['GHI'] < 10]  # Adjust threshold as needed

if not low_ghi_days.empty:
    print(f"Found {len(low_ghi_days)} records with very low GHI during daytime")
    print("These could be cloudy days, rainy days, or data issues")
    
    # Show the dates with lowest GHI
    low_ghi_summary = low_ghi_days.groupby(low_ghi_days['Timestamp'].dt.date).size()
    print("\nDates with most low-GHI records:")
    print(low_ghi_summary.sort_values(ascending=False).head(10))
else:
    print("No significant anomalies detected in daytime GHI values")

# Cleaning impact analysis

In [ ]:
## Cleaning Impact Analysis
print("=== CLEANING IMPACT ===")

# Create cleaning flag
df_clean['had_outlier'] = False
outlier_cols = [f'{col}_outlier' for col in available_columns if f'{col}_outlier' in df_clean.columns]
if outlier_cols:
    df_clean['had_outlier'] = df_clean[outlier_cols].any(axis=1)

# Compare sensor readings
sensor_cols = ['ModA', 'ModB']
available_sensors = [col for col in sensor_cols if col in df_clean.columns]

if available_sensors and 'had_outlier' in df_clean.columns:
    sensor_comparison = df_clean.groupby('had_outlier')[available_sensors].mean()
    print("Average sensor readings by outlier status:")
    print(sensor_comparison)
    
    # Plot comparison
    sensor_comparison.plot(kind='bar', figsize=(10, 6))
    plt.title('Sierra Leone - Sensor Readings: Clean vs Outlier Data')
    plt.ylabel('Average Value')
    plt.xticks(rotation=0)
    plt.grid(True, alpha=0.3)
    plt.show()

# Correlation & Relationship analysis

In [ ]:

print(" CORRELATION ANALYSIS ")

# Define variables for correlation analysis included Tamb,RH,WS,WSgust and WD as they are used in scatter plots
correlation_vars = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB', 'Tamb', 'RH', 'WS', 'WSgust', 'WD']

# Check which variables exist in our dataset
available_corr_vars = [var for var in correlation_vars if var in df_clean.columns]
print(f"Variables available for correlation: {available_corr_vars}")

# Select only numeric columns for correlation
corr_data = df_clean[available_corr_vars].select_dtypes(include=[np.number])

print(f"Data shape for correlation: {corr_data.shape}")

In [ ]:
# Create correlation heatmap
print("\nCORRELATION HEATMAP ")

plt.figure(figsize=(12, 10))

# Calculate correlation matrix
correlation_matrix = corr_data.corr()

# Create heatmap
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))  # Mask upper triangle
sns.heatmap(correlation_matrix, 
            mask=mask,
            annot=True, 
            cmap='RdBu_r', 
            center=0,
            square=True,
            fmt='.2f',
            cbar_kws={'shrink': 0.8})

plt.title('Correlation Heatmap of Solar and Environmental Variables', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

In [ ]:
## Scatter Plot Relationships
print("SCATTER PLOT RELATIONSHIPS")

# Wind vs GHI
wind_vars = ['WS', 'WSgust', 'WD']
available_wind = [var for var in wind_vars if var in df_clean.columns]

if available_wind and 'GHI' in df_clean.columns:
    fig, axes = plt.subplots(1, len(available_wind), figsize=(15, 5))
    if len(available_wind) == 1:
        axes = [axes]
    
    for i, wind_var in enumerate(available_wind):
        axes[i].scatter(df_clean[wind_var], df_clean['GHI'], alpha=0.5)
        axes[i].set_xlabel(wind_var)
        axes[i].set_ylabel('GHI')
        axes[i].set_title(f'GHI vs {wind_var}')
        axes[i].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

In [ ]:
## RH Relationships
print("HUMIDITY RELATIONSHIPS")

if 'RH' in df_clean.columns:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    
    # RH vs Tamb
    if 'Tamb' in df_clean.columns:
        ax1.scatter(df_clean['RH'], df_clean['Tamb'], alpha=0.5)
        ax1.set_xlabel('RH')
        ax1.set_ylabel('Tamb')
        ax1.set_title('Temperature vs Humidity')
        ax1.grid(True, alpha=0.3)
    
    # RH vs GHI
    if 'GHI' in df_clean.columns:
        ax2.scatter(df_clean['RH'], df_clean['GHI'], alpha=0.5)
        ax2.set_xlabel('RH')
        ax2.set_ylabel('GHI')
        ax2.set_title('Solar Radiation vs Humidity')
        ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Wind and Distribution Analysis

In [ ]:
## Wind Distribution Analysis
print("=== WIND DISTRIBUTION ===")

if 'WS' in df_clean.columns and 'WD' in df_clean.columns:
    # Simple wind direction histogram
    plt.figure(figsize=(10, 6))
    plt.hist(df_clean['WD'].dropna(), bins=36, alpha=0.7, edgecolor='black')
    plt.xlabel('Wind Direction (degrees)')
    plt.ylabel('Frequency')
    plt.title('Wind Direction Distribution')
    plt.grid(True, alpha=0.3)
    plt.show()

In [ ]:
## Variable Distributions
print("=== VARIABLE DISTRIBUTIONS ===")

# Histograms for GHI and WS
dist_vars = ['GHI', 'WS']
available_dist = [var for var in dist_vars if var in df_clean.columns]

for var in available_dist:
    plt.figure(figsize=(10, 6))
    plt.hist(df_clean[var].dropna(), bins=50, alpha=0.7, edgecolor='black')
    plt.xlabel(var)
    plt.ylabel('Frequency')
    plt.title(f'Distribution of {var}')
    plt.grid(True, alpha=0.3)
    plt.show()

# Temperature analysis

In [ ]:
## Temperature-Humidity Analysis
print("TEMPERATURE-HUMIDITY ANALYSIS")

if 'Tamb' in df_clean.columns and 'RH' in df_clean.columns:
    # Scatter plot
    plt.figure(figsize=(10, 6))
    plt.scatter(df_clean['RH'], df_clean['Tamb'], alpha=0.5)
    plt.xlabel('Relative Humidity (%)')
    plt.ylabel('Temperature (°C)')
    plt.title('Temperature vs Humidity')
    plt.grid(True, alpha=0.3)
    
    # Add correlation
    corr = df_clean['RH'].corr(df_clean['Tamb'])
    plt.text(0.05, 0.95, f'Correlation: {corr:.3f}', 
             transform=plt.gca().transAxes, 
             bbox=dict(boxstyle="round", facecolor="white"))
    plt.show()
    

# Bubble chart analysis

In [ ]:
## Bubble Chart
print("=== BUBBLE CHART ===")

if 'GHI' in df_clean.columns and 'Tamb' in df_clean.columns:
    # Use RH for bubble size if available, otherwise BP
    if 'RH' in df_clean.columns:
        bubble_var = 'RH'
        bubble_label = 'Relative Humidity'
    elif 'BP' in df_clean.columns:
        bubble_var = 'BP'
        bubble_label = 'Barometric Pressure'
    else:
        bubble_var = None
    
    if bubble_var:
        # Sample data for better visualization
        sample_data = df_clean.iloc[::10]  # Every 10th point
        
        plt.figure(figsize=(12, 8))
        scatter = plt.scatter(sample_data['Tamb'], sample_data['GHI'],
                             s=sample_data[bubble_var]/2,  # Scale bubble size
                             alpha=0.6, 
                             c=sample_data[bubble_var],
                             cmap='viridis')
        
        plt.xlabel('Temperature (°C)')
        plt.ylabel('GHI (W/m²)')
        plt.title(f'GHI vs Temperature\n(Bubble Size = {bubble_label})')
        plt.colorbar(scatter, label=bubble_label)
        plt.grid(True, alpha=0.3)
        plt.show()

# Final Summary

In [ ]:
# Final Summary for Sierraleone EDA
print(" Sierraleone EDA COMPLETED SUCCESSFULLY!")

print("\n ANALYSIS COMPLETED:")
completed_analyses = [
    "Data Loading & Initial Exploration",
    "Missing Values & Outlier Detection", 
    "Data Cleaning & Export",
    "Time Series Analysis",
    "Cleaning Impact Assessment",
    "Correlation & Relationship Analysis",
    "Wind & Distribution Analysis",
    "Temperature-Humidity Analysis",
    "Bubble Chart Visualization"
]

for analysis in completed_analyses:
    print(f"  {analysis}")









In [ ]:
## Export notebook state for reporting
print("\n=== EXPORTING ANALYSIS METRICS ===")

# Create summary statistics for reporting
analysis_metrics = {
    'total_records': len(df_clean),
    'analysis_period_days': (df_clean['Timestamp'].max() - df_clean['Timestamp'].min()).days,
    'data_quality_score': (len(df_clean) - df_clean.isna().sum().sum()) / len(df_clean) * 100,
    'avg_ghi': df_clean['GHI'].mean() if 'GHI' in df_clean.columns else None,
    'avg_temperature': df_clean['Tamb'].mean() if 'Tamb' in df_clean.columns else None,
    'avg_humidity': df_clean['RH'].mean() if 'RH' in df_clean.columns else None,
}

print("Sierra Leone Analysis Metrics for Reporting:")
for metric, value in analysis_metrics.items():
    if value is not None:
        if isinstance(value, float):
            print(f"  {metric.replace('_', ' ').title()}: {value:.2f}")
        else:
            print(f"  {metric.replace('_', ' ').title()}: {value}")

print("\n✅ Sierra Leone Exploratory Data Analysis COMPLETE!")